In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
import xml.etree.ElementTree as et
import xml.dom.minidom as Dom

def get_dic(string):
    #get dict from xml
    train_labels = et.parse(string)
    root = train_labels.getroot()
    dic = {}
    for child in root:
        key = child.attrib['source']
        sub_dic = {}
        for sub_child in child:
            sub_key =  sub_child.attrib['name']
            value = []
            for sub_sub_child in sub_child:
                value.append(sub_sub_child.attrib['id']+','+sub_sub_child.attrib['judgment'])
            sub_dic[sub_key]= value
        dic[key] = sub_dic    
    return dic



corpus = []
dom = Dom.parse('data/textual/questionable_stripped_no_fam.xml')
root = dom.documentElement
item_list = root.getElementsByTagName('text')

for item in item_list:
    #print(item.firstChild.data)
    corpus.append(item.firstChild.data)
print(len(corpus))


tfidf_vec = TfidfVectorizer(max_features=30000) 
tfidf_matrix = tfidf_vec.fit_transform(corpus)
print(tfidf_matrix)
tfidf_matrix_array = tfidf_matrix.toarray()
print(tfidf_matrix_array[0])
print(len(tfidf_matrix_array[0]))

stop_words = tfidf_vec.get_stop_words()
print(stop_words)

train_dic = get_dic('data/Obesity_data/train_groundtruth.xml')
#print(train_dic) 
test_dic = get_dic('data/Obesity_data/test_groundtruth.xml')
#print(test_dic) 


doc = Dom.Document() 
root_node = doc.createElement("diseaseset")
doc.appendChild(root_node) 


f = open('results.txt','w')
for key in train_dic:
    train_sub_dic = train_dic[key]
    test_sub_dic = test_dic[key]
    source_node = doc.createElement("diseases")
    source_node.setAttribute("source", key)
    for sub_key in train_sub_dic:
        disease_node = doc.createElement("disease")
        disease_node.setAttribute("name", sub_key)
        train_data_X = []
        train_data_Y = [] 
        train_docs = train_sub_dic[sub_key]
        print(len(train_docs))
        for train_doc in train_docs:
            temp = train_doc.split(',')
            #print(key,sub_key,temp)
            train_data_X.append(tfidf_matrix_array[int(temp[0])-1])
            train_data_Y.append(temp[1])
        clf = svm.SVC(decision_function_shape='ovr',class_weight="balanced",kernel='linear')
        #clf = GaussianNB()
        #clf = linear_model.LogisticRegression(C=1e5, class_weight = 'balanced')
        clf.fit(train_data_X, train_data_Y)
    
        test_docs = test_sub_dic[sub_key]
        test_data_X = []
        test_data_Y = [] 
        print(len(test_docs))
    
        for test_doc in test_docs:
            temp = test_doc.split(',')
            test_data_X.append(tfidf_matrix_array[int(temp[0])-1])
            test_data_Y.append(temp[1])
        
        predict_y = clf.predict(test_data_X)
        print(predict_y)
    
        correct_count = 0
        for i in range(len(test_data_Y)):
            if predict_y[i] == test_data_Y[i]:
                correct_count += 1
            doc_node = doc.createElement("doc")
            doc_node.setAttribute("id", test_docs[i].split(',')[0])
            doc_node.setAttribute("judgment", predict_y[i])
            disease_node.appendChild(doc_node) 
        accuracy = correct_count * 1.0/ len(test_data_Y)
        print(key + ','+ sub_key + ' : Accuaracy :'+ str(accuracy))
        f.write(key + ','+ sub_key + ' : Accuaracy :'+ str(accuracy) +'\n')
    
        source_node.appendChild(disease_node) 
        root_node.appendChild(source_node) 
    
f.close()
f = open("test_predict_bow_filter_questionable.xml", "w") 
f.write(doc.toprettyxml(indent = "", newl = "\n", encoding = "utf-8")) 
f.close()

1249
  (0, 3853)	0.03266314822636777
  (0, 22912)	0.005008563056308991
  (0, 5182)	0.03266314822636777
  (0, 6005)	0.03725244813487099
  (0, 840)	0.0263282719993775
  (0, 1927)	0.021736096747021028
  (0, 1701)	0.0373988165201375
  (0, 937)	0.050085630563089914
  (0, 0)	0.010017126112617982
  (0, 8186)	0.005008563056308991
  (0, 8316)	0.03914862010709666
  (0, 25137)	0.01607151691034295
  (0, 12663)	0.005142565845175196
  (0, 7854)	0.030171824905257605
  (0, 12013)	0.010017126112617982
  (0, 23827)	0.005008563056308991
  (0, 25733)	0.035059941394162936
  (0, 12679)	0.030051378337853946
  (0, 1688)	0.012103242098358063
  (0, 8970)	0.010696794799862401
  (0, 24935)	0.007077571081013522
  (0, 18082)	0.03725244813487099
  (0, 22569)	0.007513923652598066
  (0, 12480)	0.006196806955455799
  (0, 8304)	0.26044527892806757
  :	:
  (1248, 16632)	0.03125153886858629
  (1248, 11460)	0.0341953951197546
  (1248, 25261)	0.027452347372790502
  (1248, 7786)	0.03164914048949862
  (1248, 12359)	0.03487417

507
['Y' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'Y' 'Y'
 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'Y' 'U' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U'
 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y'
 'U' 'U' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'U' 'Y'
 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'Y'
 'Y' 'Y' 'U' 'U' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'Y'
 'U' 'U' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'U'
 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U'
 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'Y' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y'
 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'Y' 'U' 'U

501
['U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'U'
 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'U'
 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'U'
 'U' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'U' 'U' 'Y' 'U' 'Y' 'U'
 'U' 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y'
 'U' 'U' 'U' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y'
 'Y' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'U'
 'Y' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'Y'
 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'U'
 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'U'
 'Y' 'U' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y'
 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'Y

497
['U' 'Y' 'U' 'U' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'U' 'U'
 'U' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y'
 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U'
 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'U' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'U'
 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'U'
 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'U' 'U' 'U' 'U' 'Y'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y' 'U' 'U' 'U'
 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'Y'
 'U' 'Y' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'U' 'U'
 'Y' 'Y' 'U' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'Y'
 'U' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'Y'
 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'Y

500
['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'Y' 'N' 'N'
 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N'
 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N'
 'Y' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'Y' 'N' 'N' 'N'
 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N'
 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N'
 'N' 'Y' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'N

431
['Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N'
 'N' 'Y' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'N'
 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'Y'
 'N' 'N' 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'N'
 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'N'
 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y'
 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N'
 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'N' 'N'
 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'Y'
 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y'
 'N' 'Y' 'Y' 'N' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'N'
 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'N' 'N'
 'N' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'N' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y'
 'N' 'Y' 'N' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N

495
['N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N'
 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'Y' 'N'
 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'Y'
 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N

479
['Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'N' 'Y' 'Y' 'N'
 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N'
 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y'
 'N' 'N' 'Y' 'N' 'N' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'N'
 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N'
 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y'
 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N'
 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y'
 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y'
 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y'
 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'Y' 'N